### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2021 ###

# Transportation Optimization Model based on Minimum Cost Network Flow #
# Project proposal #

#### Boyuan Lu (blu38@wisc.edu), Junda Chen (jchen693@wisc.edu)

### Table of Contents

1. [Introduction and Data Acquisition](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)


## 1. Introduction and Data Acquisition ##

The transportation network (also called traffic road network) is a common structure of pathways for the mobilized vehicle and pedestrian [add citation]. The transportation network essentially consists of two components: the intersection with multiple entries and exits, and the road either directional or bi-directional. These specific real-world structures can be interpreted and converted with Network Flow problems. The roads will be viewed as edges with specific capacities and the intersection for roads can be simplified into the nodes in Network Flow problems. Figure 1(a) shows maps of traffic maps in Isthmus, Madison, and corresponding network flow.   
![alt text](fig1.jpg "figure 1")
The objective of this study is to use the Minimum Cost Network Flow Model to simulate the traffic flow around the Isthmus area-Madison City. The traffic data will be collected from the Department of Transportation-Wisconsin, and preprocessed to construct a simplified but valid network model. The MNFP will be the principal approach to solve the optimization model. In advance, to simulate the pattern of movement of vehicles, which will include the vehicle speed, vehicle parking, traffic jamming, we will introduce the multi-period linear programming for interests of exploring.  

The source of transports data was collected between 2013 to 2014 by the Department of Transportation-Wisconsin [add citation]. The continuous data counts vary from hourly to weekly and the time series we adapt will be the weekly average data for the purpose of simplification. Furthermore, to simulate the vary pattern caused by car parking or loading, the Parking Garages & Lots Web Map from DOT will also be our data source. The details of constructing the traffic network will be illustrated in the second secession.  
![alt text](fig2.jpg "figure 2")



## 2. Mathematical model ##

### 2.1 Network constructing ###
The traffic network will be implemented in direct graph with cycles. The nodes are set up at the location where there is an intersection or traffic signal. We adapt the weekly average traffic (Figure \ref{fig:1}(b)) as the capacity for each edges. Furthermore, the demands for each nodes will be setup as $0$ since no car is allowed to stay in the intersection of road. To simplify the model, there are multiple dummy source nodes and dummy sink nodes which are manually inserted based on local traffic condition. Specifically, E Washington Ave and University Ave will be the dummy sources or sinks. To modify the network with part lots, we add couples dummy edges and nodes with infinite capacities and demands to simulate the retentive flows left in parking lots.
![alt text](fig3.jpg "figure 3")

In [1]:
################################################################################################################################
# 2.1 network constructing
################################################################################################################################
# constructing nodes
# ns and n41 are source and sink, respectively
nodes = [:n0, :n1, :n2, :n3, :n4, :n5, :n6, :n7, :n8, :n9, :n10, :n11, :n12, :n13, :n14, :n15, :n16, :n17, :n18, :n19,
    :n20, :n21, :n22, :n23, :n24, :n25, :n26, :n27, :n28, :n29, :n30, :n31, :n32, :n33, :n34, :n35, :n36, :n37, :n38,
     :n39, :n40, :n41]
# constructing edges
edges = [:e0_1, :e0_10, :e0_11, :e1_2, :e1_9, :e2_3, :e2_8, :e3_4, :e3_7, :e4_5, :e4_6, :e5_4, :e5_6, :e5_17, :e6_5, :e6_7, :e6_15, :e7_8, :e7_3,
:e7_15, :e8_9, :e8_2, :e8_14, :e9_1, :e9_10, :e9_13, :e10_12, :e10_13, :e11_12, :e11_23, :e12_11, :e12_13, :e12_22, :e13_9, :e13_10, :e13_12, :e13_21,
:e14_8, :e14_13, :e15_7, :e15_6, :e15_14, :e15_16, :e16_6, :e16_15, :e16_17, :e17_5, :e17_16, :e17_18, :e18_17, :e18_19, :e18_30, :e19_18, :e19_20, 
:e20_15, :e20_19, :e21_22, :e21_26, :e22_21, :e22_23, :e22_25, :e23_11, :e23_22, :e23_24, :e24_23, :e24_25, :e24_41, :e25_24, :e25_26, :e25_36,
:e26_25, :e26_27, :e26_35, :e26_36, :e27_28, :e28_20, :e28_32, :e29_19, :e29_30, :e30_18, :e30_29, :e30_31, :e31_30, :e31_40, :e32_29, :e32_28, :e32_40, 
:e33_32, :e33_39, :e34_33, :e34_38, :e35_26, :e35_34, :e35_37, :e36_26, :e36_35, :e36_41, :e37_35, :e37_41, :e38_37, :e38_34, :e39_33, :e39_38,
:e40_31, :e40_32, :e40_39]

# define the capacity for the edges
caps = [5350,3850,3850,6200,2150,6950,4750,6950,2250,2700,8300,2700,1100,2200,1100,8250,1250,8250,2250,
2250,8250,4750,1650,2150,10150,2150,7900,1700,1800,4200,1800,1800,7900,2150,1700, 1800,4200,
1650,3850,2250,1250, 3850,2350,8300,2350,2350,2200,2350,2200,1650,11700,2550,11700,4150,
3100,4150,4500,4200,4500,15800,7900,4200,15800,2800,2800,9350,2500,9350,2050,9350,
2050,3650,1000,1750,3650,3100,1650,8450,3450,2550,3450,1440,1400,1400,9050,1650,1500,
9050,3750,9050,6900,1000,9050,1000,1750,9050,3100,1000,18600,18600,6900,3750,22150,
700,1500,22150]

caps_res = Dict(zip(edges, caps)) # create a dictionary for the capacity restriants

# define the incident matrix
using NamedArrays
indMatrix = NamedArray(zeros(length(edges), length(nodes)), (edges, nodes), ("edges", "nodes"))
for i in edges
    str = split(String(i), "_")
    inIndex = Symbol("n"*str[1][2:end])
    outIndex = Symbol("n"*str[2])
    indMatrix[i,inIndex] = 1
    indMatrix[i, outIndex] = -1
end
indMatrix = indMatrix'

42×106 Named LinearAlgebra.Adjoint{Float64, Matrix{Float64}}
nodes ╲ edges │   :e0_1   :e0_10   :e0_11  …  :e40_31  :e40_32  :e40_39
──────────────┼────────────────────────────────────────────────────────
:n0           │     1.0      1.0      1.0  …      0.0      0.0      0.0
:n1           │    -1.0      0.0      0.0         0.0      0.0      0.0
:n2           │     0.0      0.0      0.0         0.0      0.0      0.0
:n3           │     0.0      0.0      0.0         0.0      0.0      0.0
:n4           │     0.0      0.0      0.0         1.0      1.0      1.0
:n5           │     0.0      0.0      0.0         0.0      0.0      0.0
:n6           │     0.0      0.0      0.0         0.0      0.0      0.0
:n7           │     0.0      0.0      0.0         0.0      0.0      0.0
:n8           │     0.0      0.0      0.0         0.0      0.0      0.0
:n9           │     0.0      0.0      0.0         0.0      0.0      0.0
:n10          │     0.0     -1.0      0.0         0.0      0.0      0.0
⋮  

### 2.2 variables, constraints, and objective ###
The objective of optimization is to maximize the total traffic flow or minimize the network cost in the network. The variables are the traffic flow in each edges, which have unit as cars/week. Each edges is bounded by upper limitation provided in previous session. Meanwhile, the dummy lot nodes and edges for parking lot will be setup either as infinity or a specific storage based on data feasibility. The general forms of MCNF problem is presented as following:


- $N$: the set of nodes  
- $E$: the set of (directed) edges  
- $b_i$: the supply / demand of node $i$  
- $u_{ij}$: the capacity of the road.  
- $c_{ij}$: the cost can be a metric correlated to we want to minimize for the traffic flow. For now, we can take $c_{ij} = 1 / u_{ij}$ such that roads with larger capacity is naturally preferred over small roads with less capacity.  
- $x_{ij}$: total flow on edge $(i,j) \in E$.   


Then the mathematical model of the problem can be expressed as a standard MCNF problem:

\begin{align*}
    \min_{x} & \ \sum_{(i,j) \in E} c_{ij} x_{ij} \\
    \text{s.t.} 
    & \ \sum_{j \in N} x_{kj} - \sum_{i \in N} x_{ik} = b_k  \ \forall k \in N \\
    & \ 0 \le x_{ij} \le u_{ij} \ \forall (i,j) \in E \\
\end{align*}

In [ ]:
using NamedArrays
using JuMP, Clp

In [ ]:
# adding the dummpy arcs from sink to source
edges_Dummy = vcat(edges, [:e41_0])
caps_Dummy = vcat(caps, 10000000) #adding a large number for capacity of dummy arcs

# define the incident matrix

indMatrix_Dummy = NamedArray(zeros(length(edges_Dummy), length(nodes)), (edges_Dummy, nodes), ("edges", "nodes"))
for i in edges_Dummy
    str = split(String(i), "_")
    inIndex = Symbol("n"*str[1][2:end])
    outIndex = Symbol("n"*str[2])
    indMatrix_Dummy[i,inIndex] = 1
    indMatrix_Dummy[i, outIndex] = -1
end
indMatrix_Dummy = indMatrix_Dummy' # tranpose the incident matrix


m = Model(Clp.Optimizer)
@variable(m, X[1:length(edges_Dummy)] >=0) # define all edges as variables
b = zeros(length(nodes))
b[1] = 1000
b[42] = -1000

@constraint(m, constr1, indMatrix_Dummy*X .== b) # constraint: no demand for every node
@constraint(m, constr2, X .<= caps_Dummy) # capacity limits

c = zeros(length(edges_Dummy))
c[length(edges_Dummy)] = -1
@objective(m, Min, sum(c[i]*X[i] for i in 1:length(X)))
# @objective(m, Max, X[end])
optimize!(m)
println("Max flow: ", -objective_value(m))

In [ ]:
# Print the value for each edges in the definition
for i = 1:length(X)
    if value(X[i]) > 0
        println(edges_Dummy[i],"\t= ", value(X[i]))
    end
end
println("-- Rest of the edges are all 0.")

In [ ]:
# Print the value for each edges in the definition
for i = 1:length(X)
    if value(X[i]) > 0
        println(X[i],"\t= ", value(X[i]))
    end
end
println("-- Rest of the edges are all 0.")